## Inference the trained model

In [25]:
import torch
import torch.nn as nn
import pandas as pd
import os
import joblib
from sklearn.discriminant_analysis import StandardScaler

In [26]:
# Define the model architecture.
class FullyConnectedNeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super().__init__()

        #First Layer
        self.layer_1 = nn.Linear(input_size, 128)     # Input layer to 128 neurons
        self.activation_1 = nn.ReLU()       # ReLU activation function

        # Second Layer
        self.layer_2 = nn.Linear(128, 64)    # 128 neurons to 64 neurons
        self.activation_2 = nn.ReLU()       # Another ReLU

        # Second Layer
        self.layer_3 = nn.Linear(64, 32)    # 64 neurons to 32 neurons
        self.activation_3 = nn.ReLU()       # Another ReLU

         # Third and the final Layer
        self.layer_4 = nn.Linear(32, 1)     # Output layer to 1 neurons (classes)
        

    def forward(self, x):
        x = self.layer_1(x)
        x = self.activation_1(x)

        x = self.layer_2(x)
        x= self.activation_2(x)

        x = self.layer_3(x)
        x= self.activation_3(x)

        x = self.layer_4(x)  # Final layer does not need activation for multi-class classification
        return x


In [27]:
# load the model
def load_model(model_path, input_size):

    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found: {model_path}")

    model = FullyConnectedNeuralNetwork(input_size)
    model.load_state_dict(torch.load(model_path, map_location=torch.device("cpu")))
    print("Model loaded successfully!")
    return model

In [43]:
def inference(model, input_data_tensor):
    try:
        # convert input data to tensor if not already
        
        if not isinstance(input_data_tensor, torch.Tensor):
            input_data_tensor = torch.tensor(input_data_tensor, dtype=torch.float32)
            print("Converted input data to tensor.")  

        # Later in inference stage: load scalers
        loaded_feature_scaler = joblib.load('feature_scaler.joblib')
        loaded_target_scaler = joblib.load('target_scaler.joblib')

        model.eval()

        # Perform inference (query the model)
        with torch.no_grad(): # Use no_grad() to save memory and computations during inference
            output = model(input_data_tensor)
            y_pred_original = loaded_target_scaler.inverse_transform(output.reshape(-1, 1))

        print("\nQuerying the model with input...")
        print(f"Model input: {input_data_tensor}")
        print(f"Model output: Salary ${y_pred_original.item():.2f}")

    except Exception as e:
        print(f"An error occurred: {e}")

In [31]:

def preprocess_data_for_inference(df):
    
    all_benefits = df["AdditionalBenefits"].dropna().apply(lambda x: [b.strip() for b in str(x).split(";")])
    benefits_array = [item for sublist in all_benefits for item in sublist]
    unique_benefits = list(set(benefits_array))

    # Feature engineering: Extract key benefits as binary flags
    for benefit in unique_benefits:
        df[benefit] = df["AdditionalBenefits"].apply(lambda x: 1 if benefit in str(x) else 0)

    # Drop redundant columns
    df = df.drop(columns=["AdditionalBenefits"])

    # Preprocessing pipeline
    # Categorical features will be one-hot encoded
    # Numerical features will be scaled
    # This is necessary for the ANN to work properly
    categorical_columns = ['Gender', 'RemoteOnsite', 'Industry', 'Education', 'Location', 'JobTitle']
    numerical_columns = ['ExperienceYears', 'Certifications', 'PreviousCompanies', 'Age', 'CompanySize', 'SalaryUSD']

    # --------------------------
    # 3. Encode categorical variables
    # --------------------------
    df = pd.get_dummies(df, columns=categorical_columns, drop_first=False)

    # --------------------------
    # 4. Scale numeric features
    # --------------------------
    scaler = StandardScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

    # --------------------------
    # 5. Separate features and target
    # --------------------------
    X = df.drop(columns=["SalaryUSD"])   # Features except the salary (target variable)
    y = df["SalaryUSD"]                  # Target (regression)

    print("Input features:\n", X.head().transpose())
    print("Input target:\n", y.head().transpose())

    return X

In [ ]:
# model local file path
model_path = os.path.abspath("optimized_model.pt")

# Define the data as a dictionary
data = {
    "ExperienceYears": 0.660729,
    "Certifications": 0.285029,
    "PreviousCompanies": 0.712733,
    "Age": 0.313445,
    "CompanySize": -0.501345,
    "CommuterSupport": 0,
    "HealthInsurance": 0,
    "FlexibleHours": 1,
    "Gym": 1,
    "Bonus": 0,
    "StockOptions": 0,
    "Retirement": 1,
    "Gender_Female": False,
    "Gender_Male": True,
    "Gender_Non-binary": False,
    "RemoteOnsite_Hybrid": False,
    "RemoteOnsite_Onsite": True,
    "RemoteOnsite_Remote": False,
    "Industry_Consulting": True,
    "Industry_Finance": False,
    "Industry_Healthcare": False,
    "Industry_Retail": False,
    "Industry_Tech": False,
    "Education_Bachelors": True,
    "Education_Diploma": True,
    "Education_Masters": True,
    "Education_PhD": True,
    "Location_Australia": False,
    "Location_Germany": False,
    "Location_India": True,
    "Location_Sri Lanka": False,
    "Location_Sweden": False,
    "Location_UK": False,
    "Location_USA": False,
    "JobTitle_Data Engineer": True,
    "JobTitle_Data Scientist": False,
    "JobTitle_Fullstack Developer": False,
    "JobTitle_Lead Engineer": False,
    "JobTitle_Senior Software Engineer": False,
    "JobTitle_Software Architect": False,
    "JobTitle_Software Engineer": False
}

# Create DataFrame
df = pd.DataFrame([data])

# Step 1: Preprocess the input data
# input_data = preprocess_data_for_inference(df)
# Convert DataFrame to tensor
input_data = df.astype(float)
input_data_tensor = torch.tensor(input_data.values, dtype=torch.float32)
print("Input data tensor shape:", input_data_tensor.shape)

# Step 2: Load the model
loaded_model = load_model(model_path, input_size=41)

# Step 3: Inference
inference(loaded_model, input_data_tensor)


Input data tensor shape: torch.Size([1, 41])
Model loaded successfully!

Querying the model with input...
Model input: tensor([[ 0.6607,  0.2850,  0.7127,  0.3134, -0.5013,  0.0000,  0.0000,  1.0000,
          0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,
          1.0000,  1.0000,  1.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000]])
Model output: Salary $87646.35
